In [ ]:
# 
'''
  1. read the csv file 
  2. perform the transformation
  3. do the upsert 
  4. finally insert into rds table
'''

In [ ]:
#Here's a step-by-step guide to performing these tasks using PySpark:

### **1. Read the CSV File**
### First, read a CSV file into a PySpark DataFrame.

from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder \
    .appName("ReadCSV") \
    .getOrCreate()

# Path to the CSV file and S3 bucket
input_csv_path = "s3://your-bucket-name/input-folder/input-file.csv"  # Input CSV file in S3
output_s3_path = "s3://your-bucket-name/output-folder/transformed-data/"  # Output S3 path

# Read the CSV file into a DataFrame
df = spark.read.csv("path/to/your/file.csv", header=True, inferSchema=True)


### **2. Perform the Transformation**
#Next, perform any necessary transformations on the DataFrame. For example, you might want to filter out rows 
#with null values in certain columns and add a new column.
from pyspark.sql.functions import col, lit,date_format

# Filter out rows with null values in the 'important_column'
df_filtered = df.filter(col("important_column").isNotNull())

# Add a new column 'new_column' with a constant value
df_transformed = df_filtered.withColumn("new_column", lit("some_value"))


# Assume the date column is named 'date_column'
# Extract date, month, and year from the date column
transformed_df = df.withColumn("date", date_format(col("date_column"), "dd")) \
                   .withColumn("month", date_format(col("date_column"), "MM")) \
                   .withColumn("year", date_format(col("date_column"), "yyyy"))

#**Explanation:**
#- `filter(col("important_column").isNotNull())` filters out rows where `important_column` is null.
#- `withColumn("new_column", lit("some_value"))` adds a new column named `new_column` with a constant value.

### **3. Perform the Upsert Operation**
#To perform an upsert (insert or update) operation, you'll need to match records in your DataFrame against records in the target table in RDS. 
# Let's assume we have an `id` column that can be used to match records.

# Assuming df_target is the DataFrame read from the target RDS table
df_target = spark.read.jdbc(url="jdbc:mysql://your-rds-endpoint/db_name",
                            table="target_table",
                            properties={"user": "your_username", "password": "your_password"})

# Merge the DataFrames based on the 'id' column
df_upsert = df_transformed.alias("source").join(
    df_target.alias("target"),
    on=["id"],
    how="outer"
)

# Resolve conflicts and determine updates or inserts
df_upsert_final = df_upsert.selectExpr(
    "coalesce(source.id, target.id) as id",
    "coalesce(source.column1, target.column1) as column1",
    "coalesce(source.column2, target.column2) as column2",
    # Include other columns as needed
)

#**Explanation:**
#- The `join` operation merges the source DataFrame with the target DataFrame from RDS.
#- `coalesce` is used to resolve conflicts between source and target, preferring the non-null value.


### **4. Insert into RDS Table**
#Finally, insert the upserted DataFrame back into the RDS table.

# Write the final DataFrame back to the RDS table
df_upsert_final.write.jdbc(
    url="jdbc:mysql://your-rds-endpoint/db_name",
    table="target_table",
    mode="overwrite",  # Use 'append' if you want to add new records without deleting old ones
    properties={"user": "your_username", "password": "your_password"}
)

# Save the transformed DataFrame to S3 in CSV format
transformed_df.write.mode("overwrite").csv(output_s3_path, header=True)


#**Explanation:**
#- `mode="overwrite"` replaces the existing table with the new data. Use `mode="append"` to add new records without replacing the old ones.
#- The `jdbc` method is used to write the DataFrame to an RDS table.


In [ ]:
# dataframe vs dynamic frame 

In [ ]:
## 6. Optimization Techniques
'''
Question: What are some best practices for optimizing PySpark code?
 - Use select() to project only necessary columns.
 - Cache DataFrames when they are reused multiple times using cache() or persist().
Avoid using collect() on large datasets.
Use broadcast() for small DataFrames when joining with a large DataFrame.
Consider using partitioning and bucketing for large datasets
'''

In [ ]:
# 5. Window Functions
# Question: Can you explain how to use window functions in PySpark to calculate a moving average?
# You can use the window function to define the partitioning and ordering, and then apply aggregation functions like avg() over this window.

from pyspark.sql.window import Window
from pyspark.sql.functions import avg

window_spec = Window.partitionBy("partition_col").orderBy("order_col").rowsBetween(-2, 0)
df = df.withColumn("moving_avg", avg("value_col").over(window_spec))


In [ ]:
# 4. UDFs (User Defined Functions)
# Question: How would you create and use a UDF in PySpark to apply a custom function on a DataFrame column?
# Answer: First, define the UDF using the udf decorator or function, and then apply it to the DataFrame column.

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def custom_function(value):
    return value.upper()

udf_custom_function = udf(custom_function, StringType())

df = df.withColumn('new_column', udf_custom_function(df['existing_column']))


In [ ]:
#3. Joins in PySpark
# Question: How would you perform a join between two DataFrames on multiple columns in PySpark?
# Use the join() function and specify the join condition using a list of columns or expressions.

df1 = None 
df2 = None 

df_joined = df1.join(
    df2, 
    (df1['col1'] == df2['col1']) & (df1['col2'] == df2['col2']), 
    'inner'
)

In [ ]:
#2. Handling Nulls
#Question: How would you handle missing values (nulls) in a PySpark DataFrame?
#Answer: PySpark provides several methods to handle null values:
    # Use fillna() to replace nulls with a specific value.
    # Use dropna() to remove rows with null values.
    # Use fillna() to replace nulls selectively in specific columns.

df = df.fillna({'column1': 0, 'column2': 'unknown'})
df = df.dropna(subset=['column3'])

In [ ]:
# 1. DataFrame Operations
# Question: How would you convert a DataFrame column with JSON strings into multiple columns in PySpark?
# Answer: You can use the from_json() function along with a schema to parse the JSON strings and selectExpr() 
# to extract the fields into new columns.

from pyspark.sql.functions import from_json
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType([
    StructField("field1", StringType(), True),
    StructField("field2", StringType(), True)
])

df = df.withColumn("jsonData", from_json(df.jsonColumn, schema))
df = df.selectExpr("*", "jsonData.*").drop("jsonData")
